In [1]:
!pip install -U transformers sentence-transformers faiss-cpu langchain chromadb accelerate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.4 MB/s eta 0

In [2]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "rinna/japanese-gpt-1b"

# モデルとトークナイザを読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# GPUに乗せる
model = model.to("cuda")

# テスト入力
input_text = "日本の未来について教えてください。"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")

# テキスト生成
output = model.generate(input_ids, max_new_tokens=50, do_sample=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


日本の未来について教えてください。 先生は教育界に入るまで、具体的にどのようなキャリアを積まれたのですか?


In [3]:
# 仮の知識ベース（短文複数）で試す
docs = [
    "日本は四季があり、春には桜が咲く。",
    "夏には花火大会やお祭りが各地で開催される。",
    "秋は紅葉が美しく、食欲の季節とも呼ばれる。",
    "冬には雪が降り、スキーや温泉が楽しまれる。",
    "富士山は日本の象徴とされている。",
]


In [4]:
!pip install -q sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 軽量で日本語対応の埋め込みモデル
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 文書をベクトル化
doc_embeddings = embed_model.encode(docs)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.3 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
!pip install -q sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 軽量で日本語対応の埋め込みモデル
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# 文書をベクトル化
doc_embeddings = embed_model.encode(docs)


In [7]:
def retrieve(query, k=2):
    query_vec = embed_model.encode([query])
    D, I = index.search(query_vec, k)
    return [docs[i] for i in I[0]]


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# LLMロード（すでに済んでるならスキップOK）
model_name = "rinna/japanese-gpt-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

# プロンプト生成＋出力
def rag_answer(query):
    context = retrieve(query, k=2)
    prompt = f"質問: {query}\n参考情報:\n" + "\n".join(context) + "\n答え:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output = model.generate(input_ids, max_new_tokens=100, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [9]:
query = "日本の冬にはどんな行事がありますか？"
print(rag_answer(query))


NameError: name 'index' is not defined

In [10]:
# 必ずこのセルを先に実行！
d = doc_embeddings.shape[1]  # ベクトルの次元数
index = faiss.IndexFlatL2(d)
index.add(np.array(doc_embeddings))


In [11]:
query = "日本の冬にはどんな行事がありますか？"
print(rag_answer(query))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


質問: 日本の冬にはどんな行事がありますか? 参考情報: 日本は四季があり、春には桜が咲く。 冬には雪が降り、スキーや温泉が楽しまれる。 答え:


In [12]:
import re
from collections import Counter

def simple_reward(output_text, context_texts):
    context_words = " ".join(context_texts).lower()
    output_words = re.findall(r'\w+', output_text.lower())
    matches = [word for word in output_words if word in context_words]
    score = len(matches) / (len(set(output_words)) + 1e-5)  # 過剰一致対策
    return round(score, 3)


In [14]:
def rag_answer_with_reward(query):
    context = retrieve(query, k=2)
    prompt = f"質問: {query}\n参考情報:\n" + "\n".join(context) + "\n答え:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output_ids = model.generate(input_ids, max_new_tokens=100, do_sample=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 報酬スコアを計算
    score = simple_reward(output_text, context)

    print(f"\n🧠 出力：\n{output_text}")
    print(f"\n📚 文脈：\n{context}")
    print(f"\n🏆 関連性スコア：{score}")

    return output_text, score


In [15]:
query = "冬の楽しみ方を教えて"
rag_answer_with_reward(query)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.



🧠 出力：
質問: 冬の楽しみ方を教えて 参考情報: 冬には雪が降り、スキーや温泉が楽しまれる。 秋は紅葉が美しく、食欲の季節とも呼ばれる。 答え:

📚 文脈：
['冬には雪が降り、スキーや温泉が楽しまれる。', '秋は紅葉が美しく、食欲の季節とも呼ばれる。']

🏆 関連性スコア：0.5


('質問: 冬の楽しみ方を教えて 参考情報: 冬には雪が降り、スキーや温泉が楽しまれる。 秋は紅葉が美しく、食欲の季節とも呼ばれる。 答え:', 0.5)

In [16]:
# 保存先（初期化）
learning_log = []


In [17]:
def rag_answer_learn(query, threshold=0.4):
    context = retrieve(query, k=2)
    prompt = f"質問: {query}\n参考情報:\n" + "\n".join(context) + "\n答え:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output_ids = model.generate(input_ids, max_new_tokens=100, do_sample=True)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    score = simple_reward(output_text, context)

    # ログに記録
    record = {
        "query": query,
        "context": context,
        "output": output_text,
        "score": score
    }
    learning_log.append(record)

    # 一定以上のスコアならベクトルDBに再登録（強化）
    if score >= threshold:
        new_doc = output_text  # LLM出力そのものを記憶として扱う
        docs.append(new_doc)
        new_vec = embed_model.encode([new_doc])
        index.add(np.array(new_vec))
        print(f"\n📥 知識ベースに追加：'{new_doc}'")

    print(f"\n🧠 出力：{output_text}")
    print(f"🏆 スコア：{score}")

    return output_text, score


In [18]:
query = "冬におすすめの旅行は？"
rag_answer_learn(query)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.



📥 知識ベースに追加：'質問: 冬におすすめの旅行は? 参考情報: 冬には雪が降り、スキーや温泉が楽しまれる。 秋は紅葉が美しく、食欲の季節とも呼ばれる。 答え:'

🧠 出力：質問: 冬におすすめの旅行は? 参考情報: 冬には雪が降り、スキーや温泉が楽しまれる。 秋は紅葉が美しく、食欲の季節とも呼ばれる。 答え:
🏆 スコア：0.5


('質問: 冬におすすめの旅行は? 参考情報: 冬には雪が降り、スキーや温泉が楽しまれる。 秋は紅葉が美しく、食欲の季節とも呼ばれる。 答え:', 0.5)